In [ ]:
! pip install --upgrade jupyter ipywidgets jupyter_client "tqdm[notebook]" pqdm ipykernel mysqlclient ipython-sql sqlalchemy pandas mwxml pandas-stubs

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import mwxml
import bz2
import pandas as pd
from tqdm.notebook import tqdm

database_url = 'mysql://localhost:3306/wikipedia'
engine = create_engine(database_url)

DATASETS = {
    '202510': '/Users/rickmark/Downloads/pageviews-202510-user.csv',
    '202509': '/Users/rickmark/Downloads/pageviews-202509-user.csv'
}

In [2]:
for date_stamp, path in DATASETS.items():
    pageviews_raw = pd.read_csv(path, sep=' ',  names=['project', 'page_title', 'page_id', 'access_method', 'view_count', 'view_sequence'])
    df = pageviews_raw.drop(['project', 'access_method', 'view_sequence', 'page_title'], axis=1)

    df = df[df['page_id'].notnull()]

    df = df.groupby('page_id').agg({'view_count': 'sum'}).reset_index()
    df['ds'] = f"{date_stamp[0:4]}-{date_stamp[4:6]}-01"
    df['page_id'] = df['page_id'].astype(int)
    df.to_sql(name='pageviews_raw', con=engine, if_exists='replace', index=False)

In [24]:
%%sql
SELECT * FROM mysql.wikipedia.target_entities

,page_id,view_count,entity_type
0,64618347,9081391,1
1,80285182,3058208,1
2,80669,2454138,1
3,31251727,1961751,1
4,51137169,1746647,1
...,...,...,...
995,27105762,161860,1
996,80734471,161782,1
997,21491340,161671,1
998,49650,161544,1


In [31]:
results = []
with bz2.open('/Users/rickmark/Downloads/wikipedia/enwiki-20251220-pages-articles-multistream.xml.bz2', 'rb') as f:
    dump = mwxml.Dump.from_file(f)

    for item in tqdm(dump.pages, total=64703361):
        if top_people['page_id'].isin([item.id]).any():
            current = next(reversed(list(item)))
            results.append({'page_id': item.id, 'wiki_text': current.text})


result_df = pd.DataFrame(results)

  0%|          | 0/64703361 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
result_df

""
